In [10]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [11]:
class Reviewer(TypedDict):
    review: str
    sentiment: str
    analyzer:str
    positive_response:str
    negative_response:str

In [12]:
model=ChatOpenAI()

In [13]:
def review_content(state:Reviewer)->Reviewer:
    review_text=input("Enter the review text:")
    state['review']=review_text
    return {"review":review_text}
    

In [14]:
def find_sentiment(state:Reviewer)->Reviewer:
    prompt=f"""i am providing you a review text {state['review']} from customer for our product please analyze the review and provide the sentiment of the review"""
    senti_result=model.invoke(prompt).content
    state['sentiment']=senti_result
    return {"sentiment":senti_result}

In [15]:
def analyze_sentiment(state:Reviewer)->Reviewer:
    prompt=f"""i am providing you a review text {state['review']} from customer for our product please analyze the review of the product based on this 
    three aspects. first is Tone of the review, second is the urgency of the sentiment and third is the topic  of the sentiment"""
    analyzer_result=model.invoke(prompt).content
    state['analyzer']=analyzer_result
    return {"analyzer":analyzer_result}

In [16]:
def positive_response(state:Reviewer)->Reviewer:
    prompt=f"""i am providing you a sentiment analysis of a review {state['sentiment']} please provide a warm thankyou message for  positive review {state['review']} to this review to our customer for providing a good review"""
    positive_response_result=model.invoke(prompt).content
    state['positive_response']=positive_response_result
    return {"positive_response":positive_response_result}
    

In [17]:
def negative_response(state:Reviewer)->Reviewer:
    prompt=f"""i am providing you a sentiment analysis of a review {state['analyzer']} please provide response to this review to our customer for providing a bad review based on the analyzis {state['analyzer']}"""
    negative_response_result=model.invoke(prompt).content
    state['negative_response']=negative_response_result
    return {"negative_response":negative_response_result}

In [18]:
from typing import Literal


def check_condition(state:Reviewer)->Literal['positive_response','analyzer']:
    sent = str(state.get('sentiment', '')).lower()
    if 'positive' in sent:
        return 'positive_response'
    else:
        return 'analyzer'

In [19]:
#define state
graph=StateGraph(Reviewer)
#add node
graph.add_node("review",review_content)
graph.add_node("sentiment",find_sentiment)
graph.add_node("analyzer",analyze_sentiment)
graph.add_node("positive_response",positive_response)
graph.add_node("negative_response",negative_response)
#add edge
graph.add_edge(START,"review")
graph.add_edge("review","sentiment")
graph.add_conditional_edges("sentiment",check_condition)
graph.add_edge("positive_response",END)
graph.add_edge("analyzer","negative_response")
graph.add_edge("negative_response",END)
#compile
workflow=graph.compile()

In [20]:
initial_state={}

In [21]:
final_state=workflow.invoke(initial_state)
if 'negative_response' in final_state:
    print(final_state['negative_response'])
else:
    print(final_state.get('positive_response'))

We apologize for the disappointment you have experienced with our product. We take customer feedback very seriously and are committed to ensuring that our products meet the highest quality standards. 

We would like to make things right for you and address the issues you have encountered. Please reach out to our customer service team so that we can assist you with a solution, replacement, or refund for the defective product. Your satisfaction is our top priority and we want to ensure that you have a positive experience with our brand.

Once again, we apologize for any inconvenience this may have caused and thank you for bringing these issues to our attention. We value your feedback and will take it into consideration as we continue to improve our products and services.


In [22]:
# from IPython.display import Image
# Image(workflow.get_graph().draw_mermaid_png())